In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds

import numpy as np

import matplotlib.pyplot as plt
import time

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer

print("hello")

vocab = {"A": "0", "C": "1", "G": "2", "T": "3"}
train_rawdata, test_rawdata=tfds.load('genomics_ood',split=['train', 'test'],as_supervised=True)
train_rawdata = train_rawdata.take(100000)
test_rawdata = test_rawdata.take(1000)
#for x,y in train_rawdata:
    #print(y)

def onehotify(x):
    
    
    
        
    for key in vocab.keys():
        x = tf.strings.regex_replace(x, key,vocab[key])
            #print(x)
           
    splitted_value = tf.strings.bytes_split(x)
        #print(splitted_value)
    numeric_value = tf.cast(tf.strings.to_number(splitted_value), tf.uint8)
        #print( numeric_value)
    geneseq_onehot = tf.one_hot(numeric_value, 4)
        #print(geneseq_onehot)
    geneseq_onehot = tf.reshape(geneseq_onehot, (-1,))
        #print(geneseq_onehot)
        
        
    #print(tf.one_hot(y, 10))
    

        
    return geneseq_onehot
train_ds = train_rawdata.map(lambda x, y: (onehotify(x), tf.one_hot(y, 10))) 
test_ds = test_rawdata.map(lambda x, y: (onehotify(x), tf.one_hot(y, 10)))

#for x,y in train_ds:
    #print(x,y)
    
# Shuffle data
train_ds = train_ds.shuffle(buffer_size=256)
test_ds = test_ds.shuffle(buffer_size=128)

# Batch data
train_ds = train_ds.batch(256)
test_ds = test_ds.batch(128)

# Prefetch data
train_ds = train_ds.prefetch(128)
test_ds = test_ds.prefetch(128)

for x,y in train_ds:
    print(x,y)

class Model(Model):

    def __init__(self):
        super(Model, self).__init__()

        self.hidden_layer_1 = tf.keras.layers.Dense(units=256,
                                                    activation=tf.keras.activations.sigmoid
                                                    )

        self.hidden_layer_2 = tf.keras.layers.Dense(units=256,
                                                    activation=tf.keras.activations.sigmoid
                                                    )

        self.output_layer = tf.keras.layers.Dense(units=10,
                                                  activation=tf.keras.activations.softmax
                                                  )
    
    def call(self, x):
        #print(x)
        # Forwardstep
        x = self.hidden_layer_1(x)
        x = self.hidden_layer_2(x)
        x = self.output_layer(x)
        return x
    
def train_step(model, input, target, loss_function, optimizer):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  # training_accuracy_aggregator = []
    with tf.GradientTape() as tape:
        prediction = model(input)
        #print(input)
        #print(prediction)
        #print(target)
        train_loss = loss_function(target, prediction)
        sample_training_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
        sample_training_accuracy = np.mean(sample_training_accuracy)
        gradients = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return train_loss , sample_training_accuracy

def test(model, test_data, loss_function):
  # test over complete test data
    test_accuracy_aggregator = []
    test_loss_aggregator = []

    for (input, target) in test_data:
        prediction = model(input)
        sample_test_loss = loss_function(target, prediction)
        sample_test_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
        sample_test_accuracy = np.mean(sample_test_accuracy)
        test_loss_aggregator.append(sample_test_loss.numpy())
        test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

        test_loss = np.mean(test_loss_aggregator)
        test_accuracy = np.mean(test_accuracy_aggregator)

    return test_loss, test_accuracy
    
    
tf.keras.backend.clear_session()

### Hyperparameters
num_epochs = 10
learning_rate = 0.1
running_average_factor = 0.95

# Time the training process
start = time.time()

# Initialize the model.
model = Model()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer: Adam with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.SGD(learning_rate)

# Initialize lists for later visualization.
train_losses = []
train_accuracies = []

test_losses = []
test_accuracies = []

# Testing once before we begin
test_loss, test_accuracy = test(model, test_ds, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

# Check how model performs on train data once before we begin
train_loss, train_accuracy = test(model, train_ds, cross_entropy_loss)
train_losses.append(train_loss)
train_accuracies.append(train_accuracy)


# We train for num_epochs epochs.
for epoch in range(num_epochs):
    print('Epoch: __ ' + str(epoch))

    train_dataset = train_ds.shuffle(buffer_size=128)
    test_dataset = test_ds.shuffle(buffer_size=128)

    #training (and checking in with training)
    running_average_loss = 0
    running_average_accuracy = 0
    for (input,target) in train_ds:
        train_loss, train_accuracy = train_step(model, input, target, cross_entropy_loss, optimizer)
        running_average_loss = running_average_factor * running_average_loss  + (1 - running_average_factor) * train_loss
        running_average_accuracy = running_average_factor * running_average_accuracy  + (1 - running_average_factor) * train_accuracy

    train_losses.append(running_average_loss)
    train_accuracies.append(running_average_accuracy)

    #testing
    test_loss, test_accuracy = test(model, test_ds, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

end = time.time()
duration = end - start

print('The training took ', str(int(duration / 60)), ' minutes and ', str(duration % 60), ' seconds.' )


plt.figure()
line1, = plt.plot(train_losses, linewidth = 2)
line2, = plt.plot(test_losses, linestyle = 'dashed', linewidth = 2)
plt.xlabel("Training steps (epochs)", fontsize = 12)
plt.ylabel("Loss", fontsize = 12)
plt.legend((line1,line2),("Training","Test"), fontsize = 14)
plt.title("Loss for training and test data")
plt.show()
# One plot for training and test accuracy.
plt.figure()
line1, = plt.plot(train_accuracies, linestyle = 'dashed', color = 'green', linewidth = 2)
line2, = plt.plot(test_accuracies, color = 'red', linewidth = 2)
plt.xlabel("Training steps (epochs)", fontsize = 12)
plt.ylabel("Accuracy in %", fontsize = 12)
plt.legend((line1,line2),("Training","Test"), fontsize = 14)
plt.title("Performance on training and test data")
plt.show()

